# Compute a surface for memoris

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import interpolate

In [ ]:
import pyvista as pv

In [ ]:
# get points with known elevations
df = pd.read_excel('../../../CF_data/My_compilation/Memoris.xlsx', sheet_name='ground_surface')
df.head()

In [ ]:
xx = df.X.values
yy = df.Y.values
zz = df.Z.values

In [ ]:
xy = np.array([i for i in zip(df.X.values, df.Y.values)])
z = np.array(df.Z.values)

In [ ]:
model = interpolate.CloughTocher2DInterpolator(xy, z)
xnew, ynew = np.mgrid[df.X.min():df.X.max():100j, df.Y.min():df.Y.max():100j]
xynew = np.column_stack([xnew.flat, ynew.flat])
znew = model(xynew).reshape(100,100)

In [ ]:
plt.rcParams['font.size'] = 8.

fig = plt.figure(figsize=(16,10))

ax = fig.add_subplot(111, projection='3d')
ax.scatter(xx, yy, zz, alpha=0.8, s=5., label='point data')
ax.plot_surface(xnew, ynew, znew, alpha=0.2, label='model')
plt.show()

In [ ]:
df2 = pd.read_excel('../../../CF_data/My_compilation/Memoris.xlsx', sheet_name='boreholes')

In [ ]:
xy2 = df2[['X', 'Y']].values

In [ ]:
df2['Z2'] = model(xy2)

In [ ]:
df2.query('Z!=Z')

In [ ]:
df.rename({'X': 'x', 'Y': 'y', 'Z': 'z'}, axis=1, inplace=True)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y, df.z))

In [ ]:
from geometron.vtk import gdf_to_ug

In [ ]:
vtk = gdf_to_ug(gdf)

In [ ]:
vtk.plot(jupyter_backend='panel')

In [ ]:
grid_points = np.vstack([xnew.flatten(), ynew.flatten(), znew.flatten()]).T
df_grid = pd.DataFrame(grid_points, columns=['x', 'y', 'z'])
qq = df_grid.query('z==z')
grid_points = qq.to_numpy()

In [ ]:
points = pv.PolyData(grid_points)

In [ ]:
points.plot(jupyter_backend='panel')

In [ ]:
ground_surf = points.delaunay_2d()

In [ ]:
ground_surf.plot(jupyter_backend='panel')

In [ ]:
ground_surf.save('ground_surface.vtk')